In [ ]:
import numpy as np
import tensorflow as tf
import scipy.io
from keras import backend as K
from keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects

In [ ]:
# define the filename to save the weights of the model as a .mat
filename = 'models/hast_pow'

# define the interval of the domain for the visible dynamics
lower_limit_1 = 0.01
lower_limit_2 = 0.01
lower_limit_3 = 0.01

upper_limit_1 = 1
upper_limit_2 = 1
upper_limit_3 = 1

m1 = upper_limit_1-lower_limit_1 + upper_limit_2-lower_limit_2 + upper_limit_3-lower_limit_3;

step = 0.01;
#step = 0.025;
step_size_1 = step*(upper_limit_1-lower_limit_1)/m1
step_size_2 = step*(upper_limit_2-lower_limit_2)/m1
step_size_3 = step*(upper_limit_3-lower_limit_3)/m1

# define gamma parameter
gamma = 1

# define the beta parameters
beta1 = 0;
beta2 = 0;
beta3 = 0;

# define the number of hidden species in a single hidden layer
N = 5

# define the number of epochs to use in training the model
number_of_epochs = 40

In [ ]:
# define the custom activation function for the training with particular values of the relu_offset
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma))

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})

#define data
x1_train = np.arange(lower_limit_1, upper_limit_1, step_size_1, dtype="float32")
x2_train = np.arange(lower_limit_2, upper_limit_2, step_size_2, dtype="float32")
x3_train = np.arange(lower_limit_3, upper_limit_3, step_size_3, dtype="float32")

x1v, x2v, x3v = np.meshgrid(x1_train, x2_train, x3_train, indexing='ij')
nx1 = len(x1_train)
nx2 = len(x2_train)
nx3 = len(x2_train)

r= 2.5;
k=1.5;
a1 = 4.0;
a2 = 4.0;
b1 = 3.0;
b2 = 3.0;
d1 = 0.4;
d2 = 0.6;


f1 = a1*(x1v/(1+b1*x1v));
f2 = a2*x2v/(1+b2*x2v);


g_1 = r*x1v*(1-k*x1v) -f1*x2v
g_2 = -d1*x2v+f1*x2v-f2*x3v
g_3 = -d2*x3v+ f2*x3v

y1_train = (g_1 - beta1)/x1v
y2_train = (g_2 - beta2)/x2v
y3_train = (g_3 - beta3)/x3v



x_train = np.append(np.append(x1v.reshape(-1,1), x2v.reshape(-1,1),axis=1), x3v.reshape(-1,1),axis=1)
y_train = np.append(np.append(y1_train.reshape(-1,1), y2_train.reshape(-1,1),axis=1),y3_train.reshape(-1,1) , axis=1)

# define a neural network model that corresponds to the asymptotic neural subsytem 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(3,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(3, activation=None, use_bias=False),
])

# compile model with optimizer and choice of loss function
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [ ]:
# train the neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1)

In [ ]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]


In [ ]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 'first_layer_biases':first_layer_biases, 'output_layer_weights':output_layer_weights})